  <a href="https://colab.research.google.com/github/ikitova/MLatImperial2022/main/blob/lab_02_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Linear classification for real task

We'll try to solve clients' churn task using data of mobile network operator.

We have to predict whether customer will change the mobile network operator.

The target field here is 'Churn'.

Let's transform raw data, then make a Logistic Regression model and adjust it's parameteres.

Upload data and have a look at it

In [ ]:
!wget -N https://raw.githubusercontent.com/yandexdataschool/MLatImperial2022/main/Data/telecom_churn2.csv

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

random_state=0
df = pd.read_csv('telecom_churn2.csv')
df.head()

Transform target and  some other fields

In [ ]:
d = {'Yes' : 1, 'No' : 0}
df['International plan'] = df['International plan'].map(d)
df['Voice mail plan'] = df['Voice mail plan'].map(d)
df['Churn'] = df['Churn'].astype('int64')
df.head()

Divide data to design matrix X and target vector y.

Make a train-test split

In [ ]:
#df=df.drop('State',axis=1)
df.head()
y=df['Churn']
X=df.drop('Churn',axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2,
                                                    random_state=0)

In [ ]:
X.describe()


In [ ]:
#<YOUR TURN>
# analyse feature 'Area code' and transform it if nessesary

Further we need to:
- Impute missing numeric and categorical values.

- Separate numerical and categorical fields.

- Scale numerical features

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

numeric_data = X_train.select_dtypes([np.number])
numeric_data_mean = numeric_data.mean()
X_train = X_train.fillna(numeric_data_mean)
X_test = X_test.fillna(numeric_data_mean)
X_train.describe()

Now we don't extract all the information from the data, simply because we do not use some of the features. These features in the dataset are encoded in strings, each of them represents a certain category. 

Let's first fill in missing categorical features with special category "NotGiven". Sometimes the fact that a feature has a missing value can be a good sign itself.

In [ ]:
numeric_features = numeric_data.columns
categorical = list(X_train.dtypes[X_train.dtypes == "object"].index)
X_train[categorical] = X_train[categorical].fillna("NotGiven")
X_test[categorical] = X_test[categorical].fillna("NotGiven")

In [ ]:
X_train.head()

### Categorical features encoding

Many ML algorithms do not work with categorial features and assume only numeric. If you want to transform categorial features into numeric, you may use encoding.  Two standard transformers from sklearn for working with categorical features are `OrdinalEncoder` (simply renumbers feature values with natural numbers) and `OneHotEncoder` (dummy features).

### One Hot Encoding

https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html

<img src="https://russianblogs.com/images/855/ddd65f4f342886bb411d41a33c5528e7.png" width=50%> 




A `OneHotEncoder` is a representation of categorical variables as binary vectors.

`OneHotEncoder` assigns to each feature a whole vector consisting of zeros and one unit (which stands in the place corresponding to the received value, thus encoding it).

Then, each integer value is represented as a binary vector that is all zero values except the index of the integer, which is marked with a 1.

Is it worth to apply a scaling transformer to features encoded by `OneHotEncoder`?
What's about applying  `OrdinalEncoder` in the case of a linear model?

### Pipeline

We can write more streamlined  code with Pipeline:

<img src="https://miro.medium.com/max/620/1*ONryJuHGGUZ6PUmYTMiFxQ.png" width=50%>

Model training is often presented as a sequence of some actions with training and test sets (for example, you first need to scale the sample (and for the training set you need to apply the fit method, and for the test set - transform), and then train/apply the model (for the train sample fit, and make predictions for test sample)  

The `sklearn.pipeline.Pipeline` class allows you to store this sequence of steps and correctly applies it to both training and test samples.

sklearn also has a class to make a pipeline without naming: `sklearn.pipeline.make_pipeline` 


### ColumnTransformer
https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html

<img src="https://miro.medium.com/max/537/1*BNwN3cmbLLoU9CQoJgFSKQ.png" width=30%> 


We often need to apply different sets of tranformers to different groups of columns. For instance, we would want to apply OneHotEncoder to only categorical columns but not to numerical columns. This is where ColumnTransformer comes in. This time, we will partition the dataset keeping all columns so that we have both numerical and categorical features.

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegressionCV,LogisticRegression

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error,mean_absolute_error

column_transformer = ColumnTransformer([
    ('ohe', OneHotEncoder(handle_unknown="ignore",sparse=False), categorical),
    ('scaling', StandardScaler(), numeric_features)
])
X_train_encoded=column_transformer.fit_transform(X_train)

pd.DataFrame(X_train_encoded).head()

### LogisticRegression

We've got 2 realizations of LogisticRegression:


https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html


- class sklearn.linear_model.LogisticRegression ()
- class sklearn.linear_model.LogisticRegressionCV(*,

                     Cs=10, fit_intercept=True, cv=None, dual=False, penalty='l2',
                     
                     scoring=None,  solver='lbfgs', tol=0.0001, max_iter=100,
                     
                     class_weight=None, n_jobs=None, verbose=0, refit=True, 
                     
                     intercept_scaling=1.0, multi_class='auto', random_state=None, l1_ratios=None)
                     
   - Cs - Inverse of regularization strength; must be a positive float. Like in support vector machines, smaller values specify stronger regularization.
   - penalty -{‘l1’, ‘l2’, ‘elasticnet’, ‘none’}, default=’l2’
   - solver{‘newton-cg’, ‘lbfgs’, ‘liblinear’, ‘sag’, ‘saga’}, default=’lbfgs’. Algorithm to use in the optimization problem. Default is ‘lbfgs’. 

   - cv : cvint or cross-validation generator, default=None
            The default cross-validation generator(for example,KFold or LeaveOneOut)  used is Stratified K-Folds. If an integer is provided, then it is the number of folds used. 
   - l1_ratioslist of float, default=None. The list of Elastic-Net mixing parameter
   
In addition to the standard `fit`,`predict` methods, the `predict_proba()` method is useful for classifiers  


Let's create a logistic regression with L2-regularization in Pipeline with feature transformation, find the best parameters on cross-validation on the grid of the regularization parameter С: [0.0001,0.001,0.01,0.1,1,10,100].
We'll use the LogisticRegressionCV and the number of cross-validation blocks cv=5

In [ ]:
#import warnings
#warnings.simplefilter("ignore")

pipeline = Pipeline(steps=[
    ('ohe_and_scaling', column_transformer),
    ('regression', LogisticRegressionCV(penalty='l2',Cs=[0.0001,0.001,0.01,0.1,1,10,100],
                                        cv=5,refit=True))
])

model = pipeline.fit(X_train, y_train)
y_pred = model.predict(X_test)

y_proba = model.predict_proba(X_test)
print(pd.DataFrame(y_proba[:, 1]).head())
#print(pd.DataFrame(y_proba).head())

In [ ]:
from sklearn.metrics import accuracy_score
print("1. Test accuracy = %.4f" % accuracy_score(y_pred,y_test))
print("2. C = %.4f" % model['regression'].C_)


In [ ]:
#<YOUR TURN>
#try to use here GridSearchCV and LogisticRegression  instead of LogisticRegressionCV.
#did you get the same accuracy result?

#### Feature binarization


For feature binarization, you can use the class `sklearn.preprocessing.KBinsDiscretizer`:
sklearn.preprocessing.KBinsDiscretizer(n_bins=5, *, encode='onehot', strategy='quantile', dtype=None)

       strategy(default=’quantile’):
            - uniform - 
            - quantile -  
            - kmeans - 1D k-means cluster.
            - encode: 'ordinal'
Advantages of binarization: capturing non-monotonic and non-linear dependences feature from the target.
       


Instead of `StandardScaler`, we apply the class method `sklearn.preprocessing.KBinsDiscretizer` to numerical features with splitting into 25 groups and splitting strategy 'kmeans' to numerical features.
At the same time we apply `OneHotEncoder` to categorical features.
We use `ColumnTransformer` to combine uniformely these 2 transformation for the train and test datasets.



In [ ]:
from sklearn.preprocessing import KBinsDiscretizer

In [ ]:
%%time
column_transformer = ColumnTransformer([
    ('ohe', OneHotEncoder(handle_unknown="ignore"), categorical),
    ('binner',  KBinsDiscretizer(n_bins=25, strategy='quantile'), numeric_features)
])

pipeline = Pipeline(steps=[
    ('ohe_and_scaling', column_transformer),
    ('regression',  LogisticRegressionCV(penalty='l2',Cs=[0.0001,0.001,0.01,0.1,1,10,100],cv=5,max_iter=1000,
                                       random_state=random_state))
])

model = pipeline.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Test accuracy = %.4f" % accuracy_score(y_pred,y_test))
print("C= %.4f" % model[1].C_)


In [ ]:
#solver='liblinear'

https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html

In [ ]:
#you turn 
#apply polinomial features instead of Kbindiskretizer and calculate accuracy
#compare time of running (use magic %%time)

In [ ]:
%%time
import numpy as np
from sklearn.preprocessing import PolynomialFeatures



column_transformer = ColumnTransformer([
    ('ohe', OneHotEncoder(handle_unknown="ignore"), categorical),
    ('binner',   PolynomialFeatures(2), numeric_features)
])

pipeline = Pipeline(steps=[
    ('ohe_and_scaling', column_transformer),
    ('regression',  LogisticRegressionCV(penalty='l1',Cs=[0.0001,0.001,0.01,0.1,1,10,100],cv=5,solver='saga',max_iter=1000,
                                       random_state=random_state))
])

model = pipeline.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Test accuracy = %.4f" % accuracy_score(y_pred,y_test))
print("C= %.4f" % model[1].C_)

Visualization of quantile binaization of features

In [ ]:

import seaborn as sns
import matplotlib.pyplot as plt
df['q_minutes'] = pd.qcut(df['Total intl minutes'], 11)
df['service_calls'] = pd.cut(df['Customer service calls'], 5)
df['share'] = pd.qcut(df['Total intl charge']/df['Total day charge'],11,precision=2)
df.head(10)

In [ ]:

plt.figure(figsize=(12,5))
sns.barplot(x= 'service_calls',y='Churn',data=df,color="blue",saturation=0.25)
plt.xlabel("'Total intl minutes'")



In [ ]:
plt.figure(figsize=(14,5))
sns.barplot(x= 'share',y='Churn',data=df,color="blue",saturation=0.25)
plt.xlabel("Total intl minutes")


why polynomial features extracts more information from data than KBinsDiscretizer?

Is it worth to try polynomial featues of degree more than 2?

### Стохастический градиентый спуск в задачах классификации


Решим задачу методом стохастического градиентого спуска,
применяя такую же трансформацю признаков, как в последней задаче:
    - KBinsDiscretizer с разбиением на 25 групп и стратегией разбиения 'kmeans' к численным признакам,
    - OneHotEncoder- к категориальным.
Воспользуемся классом моделей градиентного спуска
`sklearn.linear_model.SGDClassifier` с параметрами:
- learning_rate='constant'
- max_iter=20 (сколько раз каждый объект случайно выбирается для модификации весов)
- loss='log'
- alpha=0.0001 (сила регуляризации)
- penalty='l2'
-  сетка значений шага скорости обучения epsilon (learning rate): [0.001,0.01,0.05,0.1,0.2,0.5,1.0,1.5,5,10] (Возьмем достаточно большие значения для иллюстрации расходимости)

SGDRegressor(loss='squared_error', penalty='l2') решает ту же задачу, что и Ridge() (др.solver)

Просто и быстро работает.

Чувствителен к масштабу признаков, требует гиперпараметры, от которых может заметно зависеть качество (регуляризация, max_iter)


In [ ]:
results=[]
for eps in [0.001,0.01,0.05,0.1,0.2,0.5,1.0,1.5,5,10]:
    from sklearn.linear_model import SGDClassifier
    pipeline = Pipeline(steps=[
        ('ohe_and_scaling', column_transformer),
        ('regression', SGDClassifier(max_iter=20,loss='log',penalty='l2',alpha=0.001, 
                                     learning_rate='constant',eta0=eps,
                                     random_state=random_state,n_iter_no_change=20))
    ])

    model = pipeline.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(" Test accuracy = %.4f learning rate= %.4f" % (accuracy_score(y_pred,y_test), eps))
    results.append((accuracy_score(y_pred,y_test), eps))


Видим, как при  слишком большом learning rate SGD не сходится. Не достигает такого же качества как просто LogisticRegression()

In [ ]:
print("Max test accuracy = %.4f \nlearning rate= %.4f" % 
      (max(results, key = lambda i : i[0])[0],max(results, key = lambda i : i[0])[1]))

Полностью аналогично предыдущей задаче обучим модель с параметром learning_rate='adaptive'(делит eps на 5, если нет улучшения  training loss на нескольких итерациях(число задается другими параметрами). Если задать слишком большой eps, то очень возможно не справится, зависит, в частности, от параметра n_iter_no_change и др.


In [ ]:
results=[]
for eps in [1,2,3,10]:
    from sklearn.linear_model import SGDClassifier
    pipeline = Pipeline(steps=[
        ('ohe_and_scaling', column_transformer),
        ('regression', SGDClassifier(max_iter=20,loss='log',penalty='l2',alpha=0.001,
                                     learning_rate='adaptive',eta0=eps,
                                     random_state=random_state,n_iter_no_change=5 ))
    ])

    model = pipeline.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(eps,accuracy_score(y_pred,y_test),model[1].n_iter_)
    results.append((accuracy_score(y_pred,y_test), eps))

In [ ]:
%%time
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import GridSearchCV

param_grid = {
    'loss': ['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron'],
    'penalty': ['elasticnet', 'l2', 'l1'],
    'alpha': [10 ** x for x in range(-6, 1)],
    'l1_ratio': [0, 0.05, 0.1, 0.2, 0.5, 0.8, 0.9, 0.95, 1],
    'learning_rate' : ["optimal"], 
    'max_iter' : [100, 500, 1000, 1500, 2000]
}

sdgc = SGDClassifier()

gs_sgdc = GridSearchCV(sdgc, param_grid=param_grid, cv=5, scoring="accuracy", verbose=1)
gs_sgdc.fit(X_train, y_train)